<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-3

**Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

### Библиотеки

In [1]:
!pip -qq install pymorphy2

     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 8.2 MB 27.4 MB/s 


In [2]:
!pip -qq install stop_words

In [144]:
import numpy as np

import tarfile
import json

from tqdm import tqdm

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim.models import Word2Vec, FastText
from sklearn.neighbors import NearestNeighbors

### Данные:

In [4]:
!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

--2022-08-30 09:34:01--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [following]
--2022-08-30 09:34:01--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucda62b594d3e1795e191158c782.dl.dropboxusercontent.com/cd/0/inline/Br8naDJtv4XzsXUGTnGK8XzZSf2mCYz4SaTtD5aleTHGda27cXrKjXR5wSXxNHPWSElZXlRGd2I16aEJIeyyayaae69sIFlq8m7j76_HwmDt1J7Wi_x_a7zNUuIDPe-GpHMuLbW1PRS9Y49X31pjDZ30QL4Urr_MFtN7PkQ4QELYSA/file# [following]
--2022-08-30 09:34:01--  https://ucda62b594d3e1795e191158c782.dl.dropboxusercontent.com/cd/0/inline/Br8naDJtv4XzsXUGTnGK8XzZSf2mCYz4SaTtD5aleTHGda27cXrKjXR5wSX

In [5]:
data = []

tar = tarfile.open("gazeta_jsonl.tar.gz", "r:gz")
for member in tar.getmembers():
     f = tar.extractfile(member)
     if f is not None:
         content = f.read().decode('utf-8')
         content = content.split('\n')
         # последний элемент пустой, так как строка оканчивалась на \n
         content = content[:-1]
         content = [json.loads(dic) for dic in content]
         data.extend(content)

In [6]:
len(data)

63435

In [7]:
data[0].keys()

dict_keys(['url', 'text', 'title', 'summary', 'date'])

In [8]:
data[0]['url']

'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml'

In [9]:
data[0]['title']

'Прогноз не успевает за оттоком'

In [10]:
data[0]['summary']

'В 2011 году из России уйдет $80 млрд, считают в Минэкономразвития. Менее месяца назад Центробанк давал оценку $70 млрд, повысив первоначальный прогноз вдвое. Отток капитала из страны усиливается из-за кризиса в Европе, а в декабре российским компаниям выплачивать внешние долги. На движение капитала повлияли и выборы: несмотря на их предсказуемость, бизнес хочет ясности с перестановками в правительстве.'

In [11]:
data[0]['text'][:100]

'«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 мл'

In [12]:
data[0]['date']

'2011-11-30 18:33:39'

### Обработаем данные

Для поиска будем использовать краткое содержание статьи: summary

In [123]:
def preprocess_txt(text, morpher, sw, exclude):
    spls = "".join(i for i in text.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw]
    return spls

In [124]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + [''] + [' '])
exclude = set(string.punctuation)
c = 0

for dic in tqdm(data):
  spls = preprocess_txt(dic['summary'], morpher, sw, exclude)
  sentences.append(spls)
  c += 1
  if c > 100000:
    break

100%|██████████| 63435/63435 [14:21<00:00, 73.62it/s]


In [145]:
sentences[0][:10]

['2011',
 'россия',
 'уйти',
 '80',
 'млрд',
 'считать',
 'минэкономразвития',
 'месяц',
 'центробанк',
 'давать']

### Обучим модели

In [147]:
modelW2V = Word2Vec(sentences=sentences, size=300, min_count=1, window=5)

In [148]:
modelW2V.wv['телефон']@modelW2V.wv['связь']

23.908112

In [149]:
modelW2V.wv['телефон']@modelW2V.wv['ваза']

0.88626474

In [126]:
modelFT = FastText(sentences=sentences, size=300, min_count=1, window=5, workers=8)

In [127]:
modelFT.wv['телефон']@modelFT.wv['связь']

33.177773

In [128]:
modelFT.wv['телефон']@modelFT.wv['ваза']

20.208273

### Составим векторы

In [175]:
W2V_sentences_vectors = []
FT_sentences_vectors = []

for sentence in sentences:
  W2V_vector = np.zeros(300)
  W2V_c = 0
  FT_vector = np.zeros(300)
  for word in sentence:
    if word in modelW2V.wv:
      W2V_vector += modelW2V.wv[word]
      W2V_c += 1
    FT_vector += modelFT.wv[word]
  if W2V_c>0:
    W2V_vector = W2V_vector/W2V_c
  W2V_sentences_vectors.append(W2V_vector)
  FT_vector = FT_vector/len(sentence)
  FT_sentences_vectors.append(FT_vector)

In [176]:
W2V_sentences_vectors[0][:10]

array([-0.41123487, -0.05363399,  0.28094164, -0.35656693, -0.17590769,
        0.11066948,  0.00830152,  0.6231159 , -0.18351916,  0.07159789])

In [177]:
FT_sentences_vectors[0][:10]

array([ 0.04579479,  0.22406543,  0.43532565,  0.13648562, -0.43412111,
        0.29161223,  0.779927  ,  0.3801476 , -0.22268056, -0.06325385])

### Поиск

In [178]:
W2V_neigh = NearestNeighbors(n_neighbors=5)
W2V_neigh.fit(W2V_sentences_vectors)
FT_neigh = NearestNeighbors(n_neighbors=5)
FT_neigh.fit(FT_sentences_vectors)

NearestNeighbors()

In [179]:
def search(text, model, neigh, top_n=5):
  sentence = preprocess_txt(text, morpher, sw, exclude)
  text_vector = np.zeros(300)
  c = 0
  for word in sentence:
    if word in model.wv:
      text_vector += model.wv[word]
      c+=1
  if c>0:
    text_vector = text_vector/c
  result_index = neigh.kneighbors([text_vector], return_distance=False)
  result = [data[i] for i in result_index[0]]
  return result

Выведим краткое содержание для сравнения, а так можно получить доступ и ко всему тексту, если указать 'text' вместо 'summary'

Погода

In [180]:
%%time

result = search('Погода в Москве сегодня', modelW2V, W2V_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Кратковременные дожди не испортят выходные в Москве. А на следующей неделе в столичном регионе ожидается возвращение жары, правда всего лишь на несколько дней.
2. В ближайшие выходные столичный регион ожидают жара и грозы. Дожди немного понизят температуру, но к следующей среде в Москве снова будет +30° С.
3. В первую неделю октября температура в Москве будет на несколько градусов ниже нормы. Ночью ожидаются заморозки. Зато дождей не предвидится.
4. До следующих выходных погода в столице ожидается пасмурная, ветреная, с дождями и мокрым снегом. Температура будет на 3...5° С ниже климатической нормы. Потеплеет в Москве только к следующей пятнице.
5. В выходные и на следующей неделе в Москве будет жара. Температура воздуха может достигнуть +35°С. Дождей и сильного ветра не будет. Из-за антициклонального характера погоды воздух над столицей будет сильно загрязнен.
CPU times: user 67 ms, sys: 10.1 ms, total: 77.1 ms
Wall time: 56.9 ms


In [181]:
%%time

result = search('Погода в Москве сегодня', modelFT, FT_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. В первую неделю октября температура в Москве будет на несколько градусов ниже нормы. Ночью ожидаются заморозки. Зато дождей не предвидится.
2. До следующих выходных погода в столице ожидается пасмурная, ветреная, с дождями и мокрым снегом. Температура будет на 3...5° С ниже климатической нормы. Потеплеет в Москве только к следующей пятнице.
3. Погода в Москве и Подмосковье немного ухудшится: в выходные будет хоть и тепло, но местами пройдут дожди. На следующей неделе погода также будет неустойчивой, а ближе к концу апреля прогремят грозы. Температура, впрочем, порадует: она будет на несколько градусов выше нормы.
4. Кратковременные дожди не испортят выходные в Москве. А на следующей неделе в столичном регионе ожидается возвращение жары, правда всего лишь на несколько дней.
5. В ближайшие выходные столичный регион ожидают жара и грозы. Дожди немного понизят температуру, но к следующей среде в Москве снова будет +30° С.
CPU times: user 83.1 ms, sys: 60.2 ms, total: 143 ms
Wall time: 8

Политика

In [182]:
result = search('Напряженность на востоке Украине', modelW2V, W2V_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Премьер-министр Украины заявил, что Россия представляет собой опасность для Евросоюза и Украины, так как использует свой природный газ как оружие против них. Он отметил, что Европа может оказаться в полной зависимости от России из-за газа. При этом политик отмечает, что газотранспотрная система Украины может обеспечить поставку газа в Европу.
2. Украина потеряет весь транзит российского газа в Европу, если Россия введет в эксплуатацию «Северный поток-2». А еще Россия достраивает «Турецкий поток». В таком случае у Киева не будет рычагов давления на Москву. В итоге эскалация военного конфликта неизбежна, убеждены на Украине.
3. Петр Порошенко назвал Европу неполноценной без Украины и сказал, что Евросоюз и НАТО заинтересованы в украинском членстве. Ранее сообщалось, что в Кремле восприняли слова Порошенко о необходимости ввести корабли НАТО в Азовское море как попытку решить внутриполитические проблемы. Также, Москва пообещала, что не будет смотреть сквозь пальцы на размещение баз НАТ

In [183]:
result = search('Напряженность на востоке Украине', modelFT, FT_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Украина хочет, чтобы Россия вместе с украинской военной техникой из Крыма, передала под контроль Киева и сам полуостров. Об этом заявил премьер-министр Украины Владимир Гройсман. Ранее президент России Владимир Путин заявил о готовности передать Украине корабли и военные самолеты, а также посодействовать перемещению техники на украинскую территорию.
2. Украина потеряет весь транзит российского газа в Европу, если Россия введет в эксплуатацию «Северный поток-2». А еще Россия достраивает «Турецкий поток». В таком случае у Киева не будет рычагов давления на Москву. В итоге эскалация военного конфликта неизбежна, убеждены на Украине.
3. Президент Украины Петр Порошенко заявил, что украинская армия станет одной из самых сильных в Европе. Такое обещание он сделал в преддверии военного парада в Киеве, запланированного на конец августа. Между тем эксперты скептически оценивают состояние вооруженных сил Украины.
4. Киев переименует боевые действия своих силовых структур против ополченцев сам

Спорт

In [184]:
result = search('Чемпионат мира по футболу', modelW2V, W2V_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Сборная Чехии победила команду Финляндии в матче за третье место на чемпионате мира и во второй раз подряд завоевала бронзовые медали мирового первенства.
2. Сборная России без изменений в составе отправилась на шестой этап Кубка мира по биатлону, что пройдет в Антерсельве. После итальянского этапа команда начнет готовиться к чемпионату мира.
3. В Чехии стартовал чемпионат мира U-18. В первом матче сборная России уверенно победила сверстников из Латвии.
4. Сборная России победила команду Швеции в финале чемпионата мира и завоевала золотые медали. Россияне стали чемпионами мира в 21-й раз в истории этого вида спорта. Бронзовые медали остались за командой Казахстана.
5. Сборная России по пляжному футболу обеспечила выход в четвертьфинал чемпионата мира, одержав во втором матче в группе С крупную победу над командой Таити.


In [185]:
result = search('Чемпионат мира по футболу', modelFT, FT_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Мужская сборная России по волейболу потерпела первое поражение на Кубке мира, проходящем в Японии, проиграв действующим чемпионам мира бразильцам в трех сетах.
2. Сборная Чехии победила команду Финляндии в матче за третье место на чемпионате мира и во второй раз подряд завоевала бронзовые медали мирового первенства.
3. Сборная России победила команду Швеции в финале чемпионата мира и завоевала золотые медали. Россияне стали чемпионами мира в 21-й раз в истории этого вида спорта. Бронзовые медали остались за командой Казахстана.
4. Молодежная сборная США завоевала золотые медали чемпионата мира по хоккею, одержав волевую победу над шведами в финале.
5. Сборная России без изменений в составе отправилась на шестой этап Кубка мира по биатлону, что пройдет в Антерсельве. После итальянского этапа команда начнет готовиться к чемпионату мира.


Путешествия

In [186]:
result = search('Лучшие города Европы', modelW2V, W2V_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. В крымском селе Янтарное находится самое большое в Европе, а возможно и в мире, поле тюльпанов. Да-да, не в Голландии, а в Крыму.
2. За последние несколько лет в России очень сильно возрос интерес к ставкам на спортивные события, в особенности к прогнозированию именно футбольных матчей. Наиболее популярными являются встречи Лиги чемпионов и баталии отборочного турнира чемпионата мира 2014 года, констатируют аналитики крупнейшей в Европе социальной сети для спортивных прогнозистов Bettingexpert.
3. Россия оказалась на втором месте в Европе по дешевизне бензина, на первом – Казахстан. Самое дорогое топливо среди европейских стран — в Нидерландах. Но там и зарплаты совсем не те, что в странах, расположенных на постсоветском пространстве.
4. Москва — город с самой высокой в мире стоимостью проживания в отелях. Дорожают номера в Канаде, Америке, Австралии и Африке, а в Европе падают, подсчитали аналитики британской компании Hogg Robinson Group. В лидерах же падения — Ближний Восток.
5. М

In [187]:
result = search('Лучшие города Европы', modelFT, FT_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. В крымском селе Янтарное находится самое большое в Европе, а возможно и в мире, поле тюльпанов. Да-да, не в Голландии, а в Крыму.
2. Московская улица Остоженка вошла в десятку самых дорогих улиц мира. Элитное жилье в Москве продолжит дорожать, прогнозируют эксперты. В конкурентах Остоженки уже сейчас районы Патриарших прудов и Красный Октябрь.
3. Москва и Санкт-Петербург находятся в самом конце рейтинга мировых финансовых центров, занимая 68-е и 71-е места соответственно. В тройке лидеров Лондон, Нью-Йорк и Гонконг. В ближайшие годы российская столица в десятку лидеров не переместится. Инвесторов сильнее привлекают азиатские города.
4. Кроме традиционных европейских городов (Рим, Париж и Милан), российские граждане в эти новогодние праздники также посещают Стамбул и Бангкок. Но в целом их по-прежнему больше интересует пляжный туризм. В прошлом году отечественные туристы искали более дешевые туры и предпочитали самостоятельные поездки, без участия туроператора.
5. В Москве стартовал 

Запросы с опечатками

In [188]:
result = search('футтбольная лга чемпионв', modelW2V, W2V_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Вчера опять прибежала какая-то однокурсница, которую я видел пару раз, да и то мельком, и слёзно умоляла «спасти его». И я возвращал ей ее «его» к жизни до четырёх утра (!!!).
2. Термин «офисный компьютер» сегодня всё чаще означает именно «ноутбук», а не более привычный конторским труженикам со стажем светло-серый «мини-тауэр» с замызганными клавиатурой, мышкой и подслеповатым монитором на 17 дюймов.
3. Дневник ММКФ: Годар и Трюффо работают кинокритиками, сентиментальные мадьярки тоскуют по родине, Шаброль снова приглашает в «Ад».
4. Ренуар и Суриков, Моне и Репин, Писсаро и Коровин, Пикассо и Серов, Менье и Архипов — в ГМИИ им.А.С. Пушкина открылась выставка-квест «Передвижники и импрессионисты. На пути в XX век».
5. Концерт Ника Кейва с проектом Grinderman в полупустом «Крокус Сити Холле» превратился в инфернальную мистерию с надеванием заячьих ушек, похабными блюзами и красноволосыми танцовщицами.


In [189]:
result = search('футтбольная лга чемпионв', modelFT, FT_neigh)
for i in range(len(result)):
  print(f'{i+1}.', result[i]['summary'])

1. Соперник «Спартака» по Лиге чемпионов «Бенфика» разгромила «Насьональ» и сохранила за собой первую строчку в чемпионате Португалии.
2. Соперник московского «Спартака» по Лиге чемпионов «Селтик» обыграл в чемпионате Шотландии «Данди», а соперник «Зенита» «Андрелехт» — «Зюлте-Варегем» в чемпионате Бельгии.
3. Соперник «Спартака» по Лиге чемпионов словацкая «Жилина» в своем чемпионате уверенно разобралась с «Дубницей» и вышла на первое место. Соперник же «Рубина» в Лиге чемпионов «Панатинаикос» только благодаря голам Джибриля Сиссе избежал поражения от «Кавалы» в чемпионате Греции.
4. Заключительные матчи группового этапа Лиги чемпионов определят три последних участника Лиги чемпионов. Судьба «Ювентуса», «Галатасарая» и «Бенфики» находится в их руках, тогда как «Челси» должен ждать поражение туринского клуба в Донецке от «Шахтера».
5. «Бавария» за шесть туров до финиша чемпионата Германии стала чемпионом, вернув себе почетный титул спустя два сезона. Следующая задача команды Юппа Хайнк

### Вывод:

В целом мини поисковые системы получились неплохии, они конечно не улавливают контекст, но они хотя бы разграничивают тематики: погода, политики, спорт и т.д.

Стоит отметить, что word2vec по ощущениям точнее, даже если посмотреть пример с телефон/связь/ваза, видно, что word2vec сильнее различает эти слова, так как его вектор не зашумляется n-граммами слов.

Но при этом он вообще никак не может работать с опечатками и новыми словами в отличии от fasttext, это является очень серьезным минусом, поэтому я бы почти всегда выбирал fasttext, а для продакшена, просто всегда.